## Solving bidomain equations on rectangular domain with modified FitzHugh-Nagumo model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = rectangle(3, 1, 50, 25)

In [3]:
class Model(BidomainModel):
    def initial_V_m(self):
        def value(x):
            mu, alpha = 0.2, 0.03
            return -85 + 35 / (1 + np.exp((x[0] - mu) / alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector(
            [
                5 * self.x[1] ** 2 / ufl.sqrt((25 * self.x[1] ** 4 + 1)),
                1 / ufl.sqrt((25 * self.x[1] ** 4 + 1)),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

In [4]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.initial_V_m(),
    camera_direction="xy",
    zoom=0.9,
    cmap='PiYG',
    save_to="rectangle_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:36237/index.html?ui=P_0x7f295c7b6b10_0&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [6]:
model.solve(
    T=500,
    steps=500,
    save_to="rectangle_MFN.mp4",
    camera_direction="xy",
    zoom = 0.9,
    checkpoints=[100, 300, 500],
    checkpoint_file="rectangle_MFN",
)

Solving problem: 100%|██████████| 500/500 [03:40<00:00,  2.27it/s]


### Plotting fiber orientations

In [7]:
plot_vector_field(
    domain,
    lambda x: [
        5 * x[1] ** 2 / np.sqrt((25 * x[1] ** 4 + 1)),
        1 / np.sqrt((25 * x[1] ** 4 + 1)),
        0,
    ],
    0.03,
    0.1,
    camera_direction="xy",
    save_to="rectangle_fibers.pdf",
)

Widget(value='<iframe src="http://localhost:36237/index.html?ui=P_0x7f293af7b7d0_1&reconnect=auto" class="pyvi…

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('33dc7cfaa9178add2a69886036720ff2_5151d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: '33dc7cfaa9178add2a69886036720ff2_5151d'

